In [68]:
from numpy import zeros, sqrt, pi, exp, sin, cos, linspace, meshgrid, empty
from numpy.linalg import norm
from matplotlib.pyplot import plot, figure, xlabel, ylabel, xlim, ylim, axes, show, subplot
from mpl_toolkits import mplot3d

t_0 = 0; T = 1
x_0 = 0; x_N = 1
y_0 = 0; y_M = 1

N = 20
hx = 1/(N - 1/2)
M = 20
hy = 1/(M - 1/2)
J = 50
tau = 1/J

x = linspace(x_0 - hx/2, x_N, N+1)
y = linspace(y_0 - hy/2, y_M, M+1)
t = linspace(t_0, T, J+1)



u = zeros((J+1, N+1, M+1))
u1 = zeros((N+1, M+1))

for n in range(N+1):
    for m in range(M+1):
        u[0, n, m] = 5*cos(pi*x[n]/2)*cos(3*pi*y[m]/2)
        u[1, n, m] = u[0, n, m] - tau**2/2*(25/2*pi**2*cos(pi*x[n]/2)*cos(3*pi*y[m]/2) - 2*(x[n] + y[m]))

for j in range(2, J+1):
    for m in range(M+1):
        u[j, N, m] = t[j]**2*(1 + y[m])

for j in range(2, J+1):
    for n in range(N+1):
        u[j, n, M] = t[j]**2*(x[n] + 1)

k1 = [1, 0]
k = [1, 0]
def p(A, B, C, F, k, m, N):
    u = zeros(N+1)
    alpha = zeros(N)
    beta = zeros(N+1)
    
    alpha[0] = k[0]
    beta[0] = m[0]
    
    for n in range(N-1):
        alpha[n+1] = B / (C - A*alpha[n])
        beta[n+1] = (A*beta[n] + F[n+1]) / (C - A*alpha[n])
        
    beta[N] = (k[1]*beta[N-1] + m[1]) / (1 - k[1]*alpha[N-1])
    u[N] = beta[N]
    
    n = N - 1
    while n >= 0:
        u[n] = alpha[n]*u[n+1]+beta[n]
        n -=1        
    return u

A1 = B1 = (tau/hx)**2/4; C1 = 2*A1 + 1
m1 = zeros(2)
F1 = zeros(N+1)
A = B = (tau/hy)**2/4; C = 2*A + 1
mu = zeros(2)
F = zeros(M+1)

def Fn(u, n, m):
    return (u[n-1, m] - 2*u[n, m] + u[n+1, m])/hx**2 + (u[n, m-1] - 2*u[n, m] + u[n, m+1])/hy**2 + 2*(x[n] + y[m])

def Fm(u1, u, j, n, m):
    return tau**2*u1 + A*(u[j-1, n, m-1] - 2*u[j, n, m-1]) - C*(u[j-1, n, m] - 2*u[j, n, m]) + B*(u[j-1, n, m+1] - 2*u[j, n, m+1])

for j in range(1, J):
    m1[0] = -hx/tau**2*(t[j-1]**2 - 2*t[j]**2 + t[j+1]**2)
    mu[0] = -hy*t[j+1]**2
    for m in range(1, M):
        m1[1] = (t[j-1]**2 - 2*t[j]**2 + t[j+1]**2)/tau**2*(1 + y[m] - (tau/hy)**2/4*(y[m-1] - 2*y[m] + y[m+1]))
        for n in range(1, N):
            F1[n] = Fn(u[j], n, m)
        u1[:, m] = p(A1, B1, C1, F1, k1, m1, N)
        
    for n in range(1, N):
        mu[1] = t[j+1]**2*(1 + x[n])
        for m in range(1, M):
            F[m] = Fm(u1[n, m], u, j, n, m)
        u[j+1, n] = p(A, B, C, F, k, mu, M)
    u[j+1, 0] = u[j+1, 1] - t[j+1]**2*hx
    
%matplotlib qt
fig = plt.figure(figsize = (10, 10))
ax = fig.add_subplot(1, 1, 1, projection = '3d')
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('z')
ax.set_zlim([-3,5])
X, Y = np.meshgrid(x,y)

for j in range(J+1):
    surf = ax.plot_surface(Y, X, u[j], cmap='inferno')
    plt.pause(0.0001)
    surf.remove()

def g(x,y,t):
    k1=np.pi/2
    k2=np.pi*3/2
    w=(k1**2+k2**2)**0.5
    return (x+y)*t**2 + 5*np.cos(w*t)*np.cos(k1*x)*np.cos(k2*y)
    

for j in range(J):
    surf = ax.plot_surface(Y, X, g(Y,X,t[j]), cmap='inferno')
    plt.pause(0.0001)
    surf.remove()


fig = plt.figure(figsize = (10, 10))
ax = fig.add_subplot(1, 1, 1, projection = '3d')
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('z')
for j in range(J+1):
    surf = ax.plot_surface(Y, X, u[j]-g(Y,X,t[j]), cmap='inferno')
    plt.pause(0.0001)
    surf.remove()
